<a href="https://colab.research.google.com/github/JyothyVariyampat/Benchmark-ML-and-DL-Models-for-Small-Molecule-Immunity-Target-Bioactivity-Prediction./blob/main/CNN_AID1259354.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""Deepclassifier-aid1239_revised.ipynb"""

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score

from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, Dropout

from google.colab import drive


In [ ]:
# Ensure reproducibility
import random
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Load the dataset
#df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1239/AID1239_extracted_descriptor_data.csv')
#/content/drive/MyDrive/Datasets/AID_1239/concatenated_AID_1239_Active_Inactive.csv
df = pd.read_csv('/content/drive/MyDrive/Datasets/AID_1259354/preprocessed_AID_1259354_12825Columns_Active_Inactive_activity.csv')
df_copy = df.copy()


<ipython-input-7-4e224f21785c>:4: DtypeWarning: Columns (3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264

In [ ]:
df.shape

(3648, 12826)

In [ ]:
# prompt: value counts for PUBCHEM_ACTIVITY_OUTCOME

df['PUBCHEM_ACTIVITY_OUTCOME'].value_counts()

,count
PUBCHEM_ACTIVITY_OUTCOME,
Inactive,1825
Active,1823


In [ ]:
# EDA 1
# Calculate the percentage of NaN values in each column


nan_percentage = df.isna().mean() * 100

# Find columns where more than 50% of the values are NaN
columns_with_nan_above_50 = nan_percentage[nan_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% NaN values: {len(columns_with_nan_above_50)}")
print("Columns:", columns_with_nan_above_50)


Number of columns with more than 50% NaN values: 0
Columns: []


In [ ]:
# EDA 2
# Calculate the percentage of zero values in each column



zero_percentage = (df == 0).mean() * 100

# Find columns where more than 50% of the values are 0
columns_with_zero_above_50 = zero_percentage[zero_percentage > 50].index.tolist()

print(f"Number of columns with more than 50% zero values: {len(columns_with_zero_above_50)}")
print("Columns:", columns_with_zero_above_50)


Number of columns with more than 50% zero values: 41
Columns: ['morgan_counts_779590789', 'morgan_counts_311492765', 'morgan_counts_2992053527', 'morgan_counts_3181448036', 'morgan_counts_1074748344', 'atom_pairs_((O,2,0),3,(S,2,1))', 'atom_pairs_((N,3,0),6,(N,1,2))', 'atom_pairs_((C,3,1),11,(C,2,2))', 'bpf_((B,2,0),7,(B,4,0))', 'torsions_((O,2,0),(C,4,0),(C,3,1),(O,2,1))', 'morgan_counts_3648038091', 'morgan_counts_2748413307', 'morgan_counts_3947092678', 'morgan_counts_2822513410', 'atom_pairs_((N,3,0),11,(N,3,1))', 'atom_pairs_((N,3,0),7,(N,1,2))', 'atom_pairs_((C,4,0),15,(N,3,1))', 'atom_pairs_((C,3,1),14,(S,2,1))', 'morgan_counts_758552268', 'morgan_counts_339644302', 'atom_pairs_((C,4,0),20,(C,3,1))', 'morgan_counts_1243702023', 'morgan_counts_3764343670', 'morgan_counts_725634252', 'morgan_counts_2232749444', 'morgan_counts_711780961', 'atom_pairs_((N,2,1),5,(Br,1,0))', 'atom_pairs_((N,2,1),4,(S,2,0))', 'atom_pairs_((O,1,1),9,(S,2,1))', 'atom_pairs_((O,2,0),7,(S,4,0))', 'bpf_((B

In [ ]:
# prompt: # # Replace hyphens with NaN

# Replace hyphens with NaN
df = df.replace('-', np.nan)

In [ ]:
# Remove all the columns which falls in this categories:
# Number of columns with more than 50% NaN values:
# Number of columns with more than 50% zero values:

import pandas as pd

# Assuming 'data' is your DataFrame

# 1. Identify columns with more than 50% NaN values
threshold_nan = len(df) * 0.5
columns_with_nan = df.columns[df.isna().sum() > threshold_nan]

# 2. Identify columns with more than 50% zero values
threshold_zero = len(df) * 0.5
columns_with_zero = df.columns[(df == 0).sum() > threshold_zero]

# 3. Combine both sets of columns to drop
columns_to_drop = set(columns_with_nan).union(set(columns_with_zero))

# 4. Drop these columns from the DataFrame
data_cleaned = df.drop(columns=columns_to_drop)

# Check the shape of the new DataFrame to confirm the columns have been removed
print(f"Original shape: {df.shape}")
print(f"New shape after dropping columns: {data_cleaned.shape}")


Original shape: (3648, 12826)
New shape after dropping columns: (3648, 12785)


In [ ]:
df = data_cleaned.copy()

In [ ]:
# Step 1: Fill null values with zero
df.fillna(0, inplace=True)
df = df.round(3)


In [ ]:
# Save the preprocessed dataset for future reference
df.to_csv('/content/drive/MyDrive/Datasets/AID_1239/preprocessed_AID1239_12531Columns_Active_Inactive.csv', index=False)
print(df.head())

                                              SMILES MOLECULEID  \
0  OC(=O)C1=CC(NC2=CC(=NS(=O)(=O)C3=CC=C(F)C=C3)C...  M83369620   
1  CC1=C(C#N)C(=O)NC(=O)\C1=C/C1=CC=C(O1)C1=CC(=C...  M17196202   
2   CCN1\C(C=CC2=CC=CC=C12)=C1\N=C(OC1=O)C1=CC=CC=C1  M14780255   
3        C1CC(CCN1)NC1=NC2=C(C=CN2)C(=C1)C1=NC=CC=N1  M13249907   
4   COC1=CC=C(CCNC2=C(N=C(COC3=CC=CC=C3C)O2)C#N)C=C1   M2688527   

  PUBCHEM_ACTIVITY_OUTCOME autocorr2d0 autocorr2d1 autocorr2d2 autocorr2d3  \
0                   Active       3.877       4.196       4.296       4.257   
1                   Active       3.498       3.871       3.939       3.924   
2                   Active        3.44        3.81       3.868       3.848   
3                   Active       3.386       3.726       3.692       3.629   
4                   Active       3.544       3.844       3.862       3.701   

  autocorr2d4 autocorr2d5 autocorr2d6  ... whales_27 whales_28 whales_29  \
0       4.268       4.238       4.158  ...    -0.036

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/Datasets/AID_1239/preprocessed_AID1239_12531Columns_Active_Inactive.csv")

In [ ]:
df.head()

,SMILES,MOLECULEID,PUBCHEM_ACTIVITY_OUTCOME,autocorr2d0,autocorr2d1,autocorr2d2,autocorr2d3,autocorr2d4,autocorr2d5,autocorr2d6,...,whales_27,whales_28,whales_29,whales_30,whales_31,whales_32,"torsions_((C,3,1),(C,3,1),(C,3,1),(N,3,0))","torsions_((C,4,0),(C,3,1),(C,3,1),(C,3,1))","torsions_((C,3,1),(C,3,1),(C,3,1),(C,3,1))","torsions_((C,3,1),(C,3,1),(C,3,1),(N,2,1))"
0,OC(=O)C1=CC(NC2=CC(=NS(=O)(=O)C3=CC=C(F)C=C3)C...,M83369620,Active,3.877,4.196,4.296,4.257,4.268,4.238,4.158,...,-0.036,-0.029,0.075,0.096,0.139,0.320,4.0,0.0,32.0,3.0
1,CC1=C(C#N)C(=O)NC(=O)\C1=C/C1=CC=C(O1)C1=CC(=C...,M17196202,Active,3.498,3.871,3.939,3.924,3.772,3.641,3.437,...,-0.018,0.046,0.099,0.141,0.170,0.525,3.0,3.0,19.0,0.0
2,CCN1\C(C=CC2=CC=CC=C12)=C1\N=C(OC1=O)C1=CC=CC=C1,M14780255,Active,3.440,3.810,3.868,3.848,3.793,3.605,3.388,...,-0.039,-0.027,-0.005,0.114,0.203,0.376,5.0,0.0,21.0,3.0
3,C1CC(CCN1)NC1=NC2=C(C=CN2)C(=C1)C1=NC=CC=N1,M13249907,Active,3.386,3.726,3.692,3.629,3.639,3.531,3.375,...,-0.027,0.003,0.055,0.124,0.254,0.421,1.0,0.0,8.0,9.0
4,COC1=CC=C(CCNC2=C(N=C(COC3=CC=CC=C3C)O2)C#N)C=C1,M2688527,Active,3.544,3.844,3.862,3.701,3.589,3.629,3.625,...,-0.032,-0.021,0.053,0.085,0.142,0.360,0.0,4.0,12.0,0.0


In [ ]:
# Define target and columns to drop
target_column = 'PUBCHEM_ACTIVITY_OUTCOME'
drop_columns = ['SMILES', 'MOLECULEID']


In [ ]:
# Drop unnecessary columns
df.drop(columns=drop_columns, inplace=True)


In [ ]:
# Encode target column (Active/Inactive)
label_encoder = LabelEncoder()
df[target_column] = label_encoder.fit_transform(df[target_column])

In [ ]:
# Split the features and target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

In [ ]:
# Step 2: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# prompt: # Ensure X_train, X_test, y_train, y_test are of numeric type before checking for infinity
# # Ensure X_train and X_test are of numeric type before checking for infinity
# X_train = X_train.astype(np.float64)
# X_test = X_test.astype(np.float64)
# and for  y_train, y_test
# # Check for NaN and infinity values, replace if needed
# X_train[np.isinf(X_train)] = np.nan
# X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))
# X_test[np.isinf(X_test)] = np.nan
# X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))
# for y_train, y_test
# and finally replace all the NaN with 0 and
#  null with 0

# Ensure X_train, X_test, y_train, y_test are of numeric type before checking for infinity
X_train = X_train.astype(np.float64)
X_test = X_test.astype(np.float64)
y_train = y_train.astype(np.float64)
y_test = y_test.astype(np.float64)

# Check for NaN and infinity values, replace if needed
X_train[np.isinf(X_train)] = np.nan
X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))
X_test[np.isinf(X_test)] = np.nan
X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

y_train[np.isinf(y_train)] = np.nan
y_train = np.nan_to_num(y_train, nan=np.nanmax(y_train))
y_test[np.isinf(y_test)] = np.nan
y_test = np.nan_to_num(y_test, nan=np.nanmax(y_test))

# Replace NaN with 0
X_train = np.nan_to_num(X_train, nan=0)
X_test = np.nan_to_num(X_test, nan=0)
y_train = np.nan_to_num(y_train, nan=0)
y_test = np.nan_to_num(y_test, nan=0)

In [ ]:
# # Check for NaN and infinity values, replace if needed
# X_train[np.isinf(X_train)] = np.nan
# X_train = np.nan_to_num(X_train, nan=np.nanmax(X_train))

# X_test[np.isinf(X_test)] = np.nan
# X_test = np.nan_to_num(X_test, nan=np.nanmax(X_test))

In [ ]:
# Step 3: Scale the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
# Define CNN model
def create_model(trial=None, input_shape=None):
    # If trial is passed, use Optuna suggestions; otherwise, default parameters
    n_layers = trial.suggest_int("n_layers", 2, 4) if trial else 3
    units = trial.suggest_int("units", 32, 256, step=32) if trial else 64
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1) if trial else 0.3
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True) if trial else 1e-4

    model = Sequential()
    model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape, kernel_regularizer=l2(l2_reg)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(dropout_rate))

    model.add(Flatten())
    for _ in range(n_layers - 2):
        model.add(Dense(units, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
        units //= 2

    model.add(Dense(1, activation='sigmoid'))
    return model

# Define Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    input_shape = (X_train.shape[1], 1)
    model = create_model(trial, input_shape=input_shape)

    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='binary_crossentropy', metrics=['accuracy'])

    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                        batch_size=batch_size, callbacks=[early_stopping], verbose=0)

    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    return test_acc

# Run Optuna optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=20)

# Best parameters
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Final model with best parameters
input_shape = (X_train.shape[1], 1)
model = create_model(None, input_shape=input_shape)
optimizer = Adam(learning_rate=best_params["learning_rate"])
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train final model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                    batch_size=best_params["batch_size"], callbacks=[early_stopping], verbose=1)

# Evaluate final model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Predictions
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Results
results = {
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nFinal Model Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))

[I 2024-11-19 07:08:44,979] A new study created in memory with name: no-name-708c1669-ca24-499e-b5f8-80baeffdfda9
/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
[I 2024-11-19 07:09:18,740] Trial 0 finished with value: 0.465753436088562 and parameters: {'n_layers': 4, 'units': 256, 'dropout_rate': 0.5, 'l2_reg': 0.000753509710877902, 'learning_rate': 0.004418972093037805, 'batch_size': 32}. Best is trial 0 with value: 0.465753436088562.
[I 2024-11-19 07:09:42,067] Trial 1 finished with value: 0.465753436088562 and parameters: {'n_layers': 3, 'units': 64, 'dropout_rate': 0.4, 'l2_reg': 6.730314791891273e-05, 'learning_rate': 0.007357969811059455, 'batch_size': 32}. Best is trial 0 with value: 0.

Best hyperparameters: {'n_layers': 4, 'units': 256, 'dropout_rate': 0.5, 'l2_reg': 0.000753509710877902, 'learning_rate': 0.004418972093037805, 'batch_size': 32}
Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 9s 52ms/step - accuracy: 0.4803 - loss: 0.7838 - val_accuracy: 0.4658 - val_loss: 0.7095
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5068 - loss: 0.7048 - val_accuracy: 0.4658 - val_loss: 0.6976
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4965 - loss: 0.6970 - val_accuracy: 0.4658 - val_loss: 0.6961
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5068 - loss: 0.6947 - val_accuracy: 0.4658 - val_loss: 0.6955
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5068 - loss: 0.6942 - val_accuracy: 0.4658 - val_loss: 0.7007
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5068 - loss: 0.6956 - val_accuracy: 0.4658 - val_loss: 0.6952
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.5068 - loss

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
# Define the CNN model
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=(X_train.shape[1], 1), kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Conv1D(filters=32, kernel_size=3, activation='relu', kernel_regularizer=l2(0.01)))
model.add(MaxPooling1D(pool_size=2))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Compile the model
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Early stopping
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [ ]:
# Train the model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50, batch_size=64, callbacks=[early_stopping])


Epoch 1/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.4884 - loss: 1.9856 - val_accuracy: 0.4644 - val_loss: 1.2351
Epoch 2/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 18ms/step - accuracy: 0.4850 - loss: 1.1690 - val_accuracy: 0.4644 - val_loss: 1.0672
Epoch 3/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4972 - loss: 1.0586 - val_accuracy: 0.4644 - val_loss: 1.0344
Epoch 4/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5051 - loss: 1.0265 - val_accuracy: 0.4644 - val_loss: 1.0046
Epoch 5/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5051 - loss: 0.9973 - val_accuracy: 0.4644 - val_loss: 0.9773
Epoch 6/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5051 - loss: 0.9705 - val_accuracy: 0.4644 - val_loss: 0.9521
Epoch 7/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5051 - loss: 0.9458 - val_accuracy: 0.4644 - val_loss: 0.9290
Epoch 8/50
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.5051 - loss: 0.9231 - val_accuracy: 0.4644 - v

In [ ]:
# Evaluate the model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)



In [ ]:
# Make predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)


92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step


In [ ]:
# Calculate metrics for training data
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)


In [ ]:
# Calculate metrics for testing data
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

In [ ]:
# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


Model Evaluation Metrics:
train_accuracy: 0.5096
train_roc_auc: 0.4989
train_f1: 0.6748
train_kappa: 0.0014
train_precision: 0.5093
train_recall: 1.0000
test_accuracy: 0.4644
test_roc_auc: 0.4985
test_f1: 0.6342
test_kappa: -0.0027
test_precision: 0.4650
test_recall: 0.9971


In [ ]:
# Confusion matrix and classification report
cm = confusion_matrix(y_test, y_pred_classes)
print("\nConfusion Matrix:")
print(cm)

print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes))


Confusion Matrix:
[[  0 390]
 [  1 339]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.00      0.00      0.00       390
         1.0       0.47      1.00      0.63       340

    accuracy                           0.46       730
   macro avg       0.23      0.50      0.32       730
weighted avg       0.22      0.46      0.30       730



**Below is Code with Optuna**

In [ ]:
!pip install --quiet optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 364.4/364.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.5/233.5 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.4 MB/s eta 0:00:00


In [ ]:
import optuna
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score, f1_score, cohen_kappa_score, precision_score, recall_score
from tensorflow.keras.callbacks import EarlyStopping

# Define the Optuna objective function
def objective(trial):
    # Hyperparameter suggestions
    n_layers = trial.suggest_int("n_layers", 2, 4)
    units = trial.suggest_int("units", 32, 256, step=32)
    dropout_rate = trial.suggest_float("dropout_rate", 0.1, 0.5, step=0.1)
    l2_reg = trial.suggest_float("l2_reg", 1e-5, 1e-2, log=True)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-2, log=True)
    batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])

    # Build the model
    model = Sequential()
    model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    for _ in range(n_layers - 1):
        model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
        model.add(Dropout(dropout_rate))
        units //= 2
    model.add(Dense(1, activation='sigmoid'))

    # Compile the model
    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                        batch_size=batch_size, callbacks=[early_stopping], verbose=0)

    # Evaluate the model on test data
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

    # Calculate additional metrics on test data
    y_pred = model.predict(X_test).flatten()
    y_pred_classes = (y_pred > 0.5).astype(int)

    if len(np.unique(y_test)) < 2:
        test_roc_auc = None
    else:
        test_roc_auc = roc_auc_score(y_test, y_pred)

    test_f1 = f1_score(y_test, y_pred_classes)
    test_kappa = cohen_kappa_score(y_test, y_pred_classes)
    test_precision = precision_score(y_test, y_pred_classes)
    test_recall = recall_score(y_test, y_pred_classes)

    # Return the test accuracy as the objective to maximize
    return test_acc

# Run Optuna for hyperparameter optimization
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=30)

# Best hyperparameters from Optuna
best_params = study.best_params
print("Best hyperparameters:", best_params)

# Build the final model with the best hyperparameters
n_layers = best_params["n_layers"]
units = best_params["units"]
dropout_rate = best_params["dropout_rate"]
l2_reg = best_params["l2_reg"]
learning_rate = best_params["learning_rate"]
batch_size = best_params["batch_size"]

model = Sequential()
model.add(Dense(units, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l2(l2_reg)))
model.add(Dropout(dropout_rate))
for _ in range(n_layers - 1):
    model.add(Dense(units // 2, activation='relu', kernel_regularizer=l2(l2_reg)))
    model.add(Dropout(dropout_rate))
    units //= 2
model.add(Dense(1, activation='sigmoid'))

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the final model
history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=50,
                    batch_size=batch_size, callbacks=[early_stopping], verbose=1)

# Evaluate the final model
train_loss, train_acc = model.evaluate(X_train, y_train, verbose=0)
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)

# Predictions
y_train_pred = model.predict(X_train).flatten()
y_train_pred_classes = (y_train_pred > 0.5).astype(int)
y_pred = model.predict(X_test).flatten()
y_pred_classes = (y_pred > 0.5).astype(int)

# Train metrics
train_roc_auc = roc_auc_score(y_train, y_train_pred)
train_f1 = f1_score(y_train, y_train_pred_classes)
train_kappa = cohen_kappa_score(y_train, y_train_pred_classes)
train_precision = precision_score(y_train, y_train_pred_classes)
train_recall = recall_score(y_train, y_train_pred_classes)

# Test metrics
test_roc_auc = roc_auc_score(y_test, y_pred)
test_f1 = f1_score(y_test, y_pred_classes)
test_kappa = cohen_kappa_score(y_test, y_pred_classes)
test_precision = precision_score(y_test, y_pred_classes)
test_recall = recall_score(y_test, y_pred_classes)

# Output the results
results = {
    "train_accuracy": train_acc,
    "train_roc_auc": train_roc_auc,
    "train_f1": train_f1,
    "train_kappa": train_kappa,
    "train_precision": train_precision,
    "train_recall": train_recall,
    "test_accuracy": test_acc,
    "test_roc_auc": test_roc_auc,
    "test_f1": test_f1,
    "test_kappa": test_kappa,
    "test_precision": test_precision,
    "test_recall": test_recall
}

print("\nModel Evaluation Metrics:")
for key, value in results.items():
    print(f"{key}: {value:.4f}")


[I 2024-11-19 06:40:55,467] A new study created in memory with name: no-name-0fcd90f1-b3dd-44fe-9976-c12197cca9ef
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step


[I 2024-11-19 06:41:09,254] Trial 0 finished with value: 0.4643835723400116 and parameters: {'n_layers': 2, 'units': 128, 'dropout_rate': 0.1, 'l2_reg': 4.0066221144964603e-05, 'learning_rate': 5.8121613070106374e-05, 'batch_size': 128}. Best is trial 0 with value: 0.4643835723400116.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step


[I 2024-11-19 06:41:22,884] Trial 1 finished with value: 0.4643835723400116 and parameters: {'n_layers': 3, 'units': 192, 'dropout_rate': 0.1, 'l2_reg': 0.0006726334988625558, 'learning_rate': 0.0019810284203500748, 'batch_size': 32}. Best is trial 0 with value: 0.4643835723400116.
/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

23/23 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step


[I 2024-11-19 06:47:57,986] Trial 29 finished with value: 0.4643835723400116 and parameters: {'n_layers': 4, 'units': 64, 'dropout_rate': 0.30000000000000004, 'l2_reg': 2.6170583202256404e-05, 'learning_rate': 4.388261666235764e-05, 'batch_size': 64}. Best is trial 12 with value: 0.5369862914085388.


Best hyperparameters: {'n_layers': 4, 'units': 32, 'dropout_rate': 0.4, 'l2_reg': 0.0001020514541821569, 'learning_rate': 0.0004685633215134769, 'batch_size': 32}


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 8s 40ms/step - accuracy: 0.4936 - loss: 0.7078 - val_accuracy: 0.4658 - val_loss: 0.7026
Epoch 2/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5075 - loss: 0.7019 - val_accuracy: 0.4658 - val_loss: 0.6974
Epoch 3/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5077 - loss: 0.8804 - val_accuracy: 0.4644 - val_loss: 0.7965
Epoch 4/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5007 - loss: 1.1042 - val_accuracy: 0.4644 - val_loss: 0.7014
Epoch 5/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4838 - loss: 0.7807 - val_accuracy: 0.5342 - val_loss: 0.6979
Epoch 6/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4888 - loss: 0.7304 - val_accuracy: 0.5342 - val_loss: 0.6982
Epoch 7/50
92/92 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5139 - loss: 0.7056 - val_accuracy: 0.5342 - val_loss: 0.6986
92/92 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Model Evaluation Metrics:
train_accur